# Ch.13 Bayesian statistics
The goal of a statistical analysis of a process is to figure out what are the variables that play a role and the relationships between them. Usually the observations have a random component. We might have a model of the process, for example we might start with two variables: one dependent variable y and one independent variable x. We might make two additional assumptions: the relationship between y and x is linear and the random compnent is normally distributed. The next step is to assess our assumptions using a sample of observations.

In [ ]:
import math
import numpy as np
import scipy.stats as scs
import scipy.optimize as sco
from scipy import interpolate
import pymc as pm
import statsmodels.api as sm
import pandas as pd
from pylab import plt, mpl
import warnings
warnings.filterwarnings('ignore')
print('Matplotlib version: {}'.format(mpl.__version__))
print('NumPy version: {}'.format(np.__version__))
print('Pandas version: {}'.format(pd.__version__))
print('PyMC version: {}'.format(pm.__version__))
print('Statsmodels version: {}'.format(sm.__version__))

We can simulate a linear process by sampling the random component from a standard distribution with mean=0 and standard deviation std=1

$$y = a + bx + \epsilon$$

In [ ]:
size = 500
x = np.linspace(0, 10, size)
y = 4 + 2 * x + np.random.standard_normal(size) * 2

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c=y, marker='v', cmap='coolwarm')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')

## Ordinary least squares
Now we can fit the simulated data using the [NumPy Polynomial.fit()](https://numpy.org/doc/stable/reference/generated/numpy.polynomial.polynomial.Polynomial.fit.html) method that performs a polynomial fit using the least squares method. Since we assume the relationship is linear we set the degree of the polynomial to 1. The method returns the coefficients of the polynomial.

In [ ]:
from numpy.polynomial import Polynomial
reg = Polynomial.fit(x, y, deg=1)
reg

In [ ]:
a = reg.convert().coef[0]
b = reg.convert().coef[1]
print('Linear coefficients\na={:.2f}\nb={:.2f}'.format(a, b))

We plot the linear fit to the data

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c=y, marker='v', cmap='coolwarm')
y_fit = a + b * x
plt.plot(x, y_fit, lw=2.0)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')

## Bayes theorem
The coefficient a and b are not fixed values, they come with an associated error that can be estimated with a certain degree of confidence or accuracy. We can use the Bayesian approach to calculate the standard error of the two coefficients. Before that we discuss briefly the theory behind the Bayes theorem. Bayes' theorem is based on the multiplication theorem that can be stated as

$$P(H|D)P(D) = P(D|H)P(D)$$

so that

$$P(H|D) = \frac{P(D|H)P(H)}{P(D)}$$

where the probabilities have the following interpretation

* The hypothesis, or prior probability, P(H)
* The likelihood, or probability for the observations given the initial hypothesis, P(D|H)
* The posterivion probability, given the observations, P(H|D)
* The total probability of the observations, P(D)

In our example we assume the hypothesis that the coefficients a and b are normally distributed and independent with zero mean value and variance in [0, 20.0] and [0, 10.0] respectively, that is our prior is

$$P(H) = P(a,b) = P(a)P(b) = N(0, \sigma_a)N(0, \sigma_b)$$

For the likelihood we assume that the y(x) values are also normally distributed with mean value $\overline{y}$, calculated from the observations, and a variance $\sigma$ that we assume uniformly distributed in [0, 10.0]

$$P(D|H) = N(\overline{y}, \sigma) = \frac{1}{\sigma \sqrt{2 \pi}}e^{\frac{1}{2}(\frac{y - \overline{y}}{\sigma})^2}$$

and

$$\sigma = U(0, 10.0)$$

We use [PyMC](https://www.pymc.io/) to build the model.

In [ ]:
%%time
with pm.Model() as model:
  # model
  a = pm.Normal('a', mu=0, sigma=20)
  b = pm.Normal('b', mu=0, sigma=10)
  sigma = pm.Uniform('sigma', lower=0, upper=10)
  y_est = a + b * x
  likelihood = pm.Normal('y', mu=y_est, sigma=sigma, observed=y)
  # inference
  start = pm.find_MAP()
  step = pm.NUTS()
  trace = pm.sample(100, tune=1000, start=start, progressbar=True)

In [ ]:
pm.summary(trace)

The sampler generates a number of parallel chains with the results for the parameters depending on the number of cores available. The values in the chains can be used together by flattening the array in one single dimension.

In [ ]:
trace['posterior']

In [ ]:
pm.plot_trace(trace, lines={'alpha': 4, 'beta': 2, 'sigma': 2});

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c=y, marker='v', cmap='coolwarm')
#plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')

# Access the posterior samples for 'a' and 'b' and flatten them
a_samples = trace.posterior.a.values.flatten()
b_samples = trace.posterior.b.values.flatten()

# Iterate through each sample and plot the corresponding line
for i in range(len(a_samples)):
  y_sampled = a_samples[i] + b_samples[i] * x
  plt.plot(x, y_sampled, color='gray', alpha=0.1) # Plot with some transparency

# Optionally, plot the mean Bayesian fit
mean_a = a_samples.mean()
mean_b = b_samples.mean()
y_mean_fit = mean_a + mean_b * x
plt.plot(x, y_mean_fit, color='red', lw=2.0, label='Mean Bayesian Fit')
plt.legend()

In [ ]:
raw = pd.read_csv('source/tr_eikon_eod_data.csv', index_col=0, parse_dates=True)
data = raw[['GDX', 'GLD']].dropna()
data = data / data.iloc[0]
data.info()

In [ ]:
data.corr()

In [ ]:
data.index

In [ ]:
data.plot(figsize=(10, 6));

## Updating estimates over time

In [ ]:
mpl_dates = mpl.dates.date2num(data.index.to_pydatetime())
mpl_dates[:3]

In [ ]:
from pymc.distributions.timeseries import GaussianRandomWalk

In [ ]:
subsample_alpha = 50
subsample_beta = 50

In [ ]:
model_randomwalk = pm.Model()
with model_randomwalk:
    sigma_alpha = pm.Exponential('sig_alpha', 1. / .02, initval=.1)
    sigma_beta = pm.Exponential('sig_beta', 1. / .02, initval=.1)
    alpha = GaussianRandomWalk('alpha', sigma_alpha ** -2, shape=int(len(data) / subsample_alpha))
    beta = GaussianRandomWalk('beta', sigma_beta ** -2, shape=int(len(data) / subsample_beta))
    alpha_r = np.repeat(alpha, subsample_alpha)
    beta_r = np.repeat(beta, subsample_beta)
    regression = alpha_r + beta_r * data['GDX'].values[:2100]
    sd = pm.Uniform('sigma', 0, 20)
    likelihood = pm.Normal('GLD', mu=regression, sigma=sd, observed=data['GLD'].values[:2100])

In [ ]:
with model_randomwalk:
    start = pm.find_MAP(vars=[alpha, beta])
    step = pm.NUTS()
    trace_rw = pm.sample(250, tune=1000, start=start, progressbar=True)

In [ ]:
pm.summary(trace_rw).head()

In [ ]:
trace_rw['posterior']

In [ ]:
sh = np.shape(trace_rw['posterior']['alpha'])
sh

In [ ]:
part_dates = np.linspace(min(mpl_dates), max(mpl_dates), sh[1])
part_dates[:3]

In [ ]:
from datetime import date
#index = [date.fromordinal(int(date)) for date in part_dates]

In [ ]:
alpha = {'alpha_%i' % i: v for i, v in enumerate(trace_rw['posterior']['alpha']) if i < 20}
alpha

In [ ]:
df_alpha = pd.DataFrame(alpha['alpha_0'], index=data.index[:250])

In [ ]:
beta = {'beta_%i' % i: v for i, v in enumerate(trace_rw['posterior']['beta']) if i < 20}
beta

In [ ]:
df_beta = pd.DataFrame(beta['beta_0'], index=data.index[:250])

In [ ]:
ax = df_alpha.plot(color='b', style='-.', legend=False, lw=0.7, figsize=(10, 6))
df_beta.plot(color='r', style='-.', legend=False, lw=0.7, ax=ax)